In [ ]:
import napari
import scipy as sp
import numpy as np
import tifffile
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
import os
import glob
import cv2
import skimage as ski
import dask
import plotly.express as px
import pandas as pd

# Setup Notebook

In [2]:
viewer = napari.Viewer()

In [3]:
peak_channel = 1

In [4]:
scale = [0.20, 0.062, 0.062]

# Utility Functions

In [5]:
def backsub(inp, radius=20):
    filterSize =(radius, radius)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                    filterSize)
    blurred = cv2.GaussianBlur(inp, (5, 5), 0)
    tophat_img = cv2.morphologyEx(blurred,
                                cv2.MORPH_TOPHAT,
                                kernel)
    rtn = inp.astype(np.single) - (blurred-tophat_img)
    rtn = np.clip(rtn, 0, np.inf)
    return rtn
def backsub_3d(inp, radius=20):
    shape = inp.shape
    reshaped = inp.reshape(-1, shape[-2], shape[-1])
    process = [dask.delayed(backsub)(i, radius) for i in reshaped]
    rslt = np.array(dask.compute(*process))
    rslt = rslt.reshape(shape)
    return rslt

In [6]:
def filter_label_size(labels, min_size=10, max_size=100):
    from skimage import measure
    label_sizes = np.bincount(labels.ravel())
    too_small = label_sizes < min_size
    too_large = label_sizes > max_size
    labels[np.isin(labels, np.where(too_small | too_large))] = 0
    labels = ski.segmentation.relabel_sequential(labels)[0]
    return labels

In [7]:
def find_peaks(img, threshold=0.1, display=False):
    #LoG = -ndi.gaussian_laplace(img/1000, sigma=[2,4,4])
    LoG = -ndi.gaussian_laplace(img/1000, sigma=[1,2,2])
    max_peaks = peak_local_max(LoG, min_distance=1, threshold_rel=threshold)
    if display:
        viewer.add_image(LoG, blending='additive', colormap='magenta', scale=scale)
        viewer.add_points(max_peaks, n_dimensional=True, size=6, scale=scale, name='AllPeaks')
    return max_peaks

def pair_finder(peaks, max_dist=6, scale=[0.12, 0.04, 0.04]):
    scaled_peaks = peaks.copy().astype(float)
    scaled_peaks[:,0] = scaled_peaks[:,0] * scale[0]
    scaled_peaks[:,1] = scaled_peaks[:,1] * scale[1]
    scaled_peaks[:,2] = scaled_peaks[:,2] * scale[2]
    
    d_matrix = sp.spatial.distance.squareform(sp.spatial.distance.pdist(scaled_peaks))
    d_matrix[d_matrix==0] = 10000
    
    min_pos = np.argmin(d_matrix, axis=0)
    distances = np.min(d_matrix, axis=0)
    sorte = np.argsort(distances)
    
    return_distances = []
    lst = []
    for a in np.arange(0,len(sorte)):
        if distances[sorte[a]]<max_dist:
            if any([all (peaks[sorte[a]]==b) for b in lst]) | any([all(peaks[min_pos][sorte[a]]==b) for b in lst]):
                lst
            elif np.abs(peaks[sorte[a]][0]-peaks[min_pos][sorte[a]][0])>6:
                lst
            else:
                lst.append(peaks[sorte[a]])
                lst.append(peaks[min_pos][sorte[a]])
                return_distances.append(distances[sorte[a]])
    return lst, return_distances

def find_peaks_in_file(fname, display=False, threshold=0.16, viewer=None):
    img = tifffile.imread(fname)
    img = backsub_3d(img, radius=20)
    peaks = find_peaks(img[:,peak_channel,:,:], threshold=threshold, display=display)
    filtered_peaks, distances = pair_finder(peaks, max_dist=1.2, scale=scale)
    
    if display:
        viewer.add_image(img, channel_axis=1, scale=scale)
        viewer.add_points(filtered_peaks[::2], n_dimensional=True, size=6, scale=scale, name='FilteredPeaks', face_color='magenta')
        viewer.add_points(filtered_peaks[1::2], n_dimensional=True, size=6, scale=scale, name='FilteredPeaks', face_color='yellow')
    return filtered_peaks, distances

In [8]:
def quantify_peaks(peaks, fname, sigma=[1.0,3.0,3.0], display=False, viewer=None):
    img = tifffile.imread(fname)
    img = backsub_3d(img, radius=20)[:,peak_channel,:,:]
    blurred = ndi.gaussian_filter(img, sigma=sigma)
    pks = np.array(peaks).astype(int)
    intensities = blurred[pks[:,0], pks[:,1], pks[:,2]]
    
    df = pd.DataFrame(peaks, columns=['Z', 'Y', 'X'])
    df['Intensity'] = intensities
    df['File'] = fname
    if display:
        viewer.add_image(blurred, scale=scale)
    
    return df

# Process Data

In [9]:
fnames = glob.glob('data/*.tif')

In [10]:
fname = fnames[0]
peaks, distances = find_peaks_in_file(fname, display=True, threshold=0.16, viewer=viewer)
cdf = quantify_peaks(peaks, fname, sigma=[1.0,3.0,3.0], display=True, viewer=viewer)

In [11]:
df = pd.DataFrame()
for fname in fnames:
    print(f'Processing {fname}')
    peaks, distances = find_peaks_in_file(fname,  threshold=0.16)
    distances = [x for x in distances for _ in (0, 1)]
    cdf = quantify_peaks(peaks, fname, sigma=[1.0,3.0,3.0], )
    cdf['distance'] = distances
    df = pd.concat([df, cdf])

Processing data\image001-1.tif
Processing data\image002-1.tif
Processing data\image003-1.tif
Processing data\image004-1.tif
Processing data\image005-1.tif
Processing data\image006-1.tif
Processing data\image007-1.tif
Processing data\image009-1.tif
Processing data\image010-1.tif
Processing data\image011-1.tif
Processing data\image013-1.tif
Processing data\image014-1.tif
Processing data\image015-1.tif
Processing data\image016-1.tif
Processing data\image017-1.tif
Processing data\image018-1.tif
Processing data\image019-1.tif
Processing data\rep1_image20-1.tif
Processing data\rep1_image21-1.tif
Processing data\rep1_image22-1.tif


In [12]:
df['pair'] = df.index // 2

In [13]:
agged = df.groupby(['File', 'pair']).agg({'Intensity':['max', 'min']}).reset_index()
agged.columns = ['File', 'pair', 'max_intensity', 'min_intensity']
agged['Ratio'] = agged['max_intensity'] / agged['min_intensity']

# Analyze


In [14]:
df['med_intensity'] = df.groupby('File')['Intensity'].transform('median')
df['normed_intensity'] = df['Intensity'] / df['med_intensity']
df['pid'] = 'First'
df.iloc[1::2, df.columns.get_loc('pid')] = 'Second'

In [15]:
px.violin(df, x='File', y='Intensity', points='all')

In [16]:
px.box(df, x='File', y='normed_intensity', points='all')

In [17]:
df['low_ptile'] = df.groupby('File')['normed_intensity'].transform(np.percentile, q=10)
df['high_ptile'] = df.groupby('File')['normed_intensity'].transform(np.percentile, q=90)
df['ratio_ptile'] = df['high_ptile'] / df['low_ptile']
tagged = df.groupby('File').agg({'ratio_ptile': 'mean', 'normed_intensity': 'mean', 'low_ptile':'mean', 'high_ptile':'mean', 'normed_intensity':'std'}).reset_index()
tagged

,File,ratio_ptile,normed_intensity,low_ptile,high_ptile
0,data\image001-1.tif,1.616369,0.198186,0.759414,1.227493
1,data\image002-1.tif,1.837496,0.240186,0.710500,1.305541
2,data\image003-1.tif,1.715107,0.223192,0.740202,1.269525
3,data\image004-1.tif,1.663831,0.284925,0.805137,1.339612
4,data\image005-1.tif,2.143216,0.297824,0.635735,1.362518
5,data\image006-1.tif,1.546455,0.165012,0.807470,1.248716
6,data\image007-1.tif,1.816840,0.297585,0.704583,1.280114
7,data\image009-1.tif,1.755976,0.283740,0.818949,1.438055
8,data\image010-1.tif,1.768996,0.239760,0.749351,1.325600
9,data\image011-1.tif,1.602277,0.190299,0.773831,1.239892


In [18]:
f = px.box(tagged, y=['low_ptile', 'high_ptile'], hover_data=['File'], points='all', range_y=[0,2], width=500, title='10th and 90th Percentiles of <br>Median Normalized Intensities')
f.write_html('percentiles.html')
f

In [19]:
tagged.iloc[:,-2:].mean()

low_ptile     0.771742
high_ptile    1.293931
dtype: float32

In [20]:
tagged.iloc[:,-2:].std()

low_ptile     0.048355
high_ptile    0.060003
dtype: float32

In [21]:
df.to_csv('quantified_peaks.csv')
tagged.to_csv('percentiles.csv')